In [18]:
# Load in libraries

import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# load in directories
from experiment_code.constants import Defaults

## Make spreadsheet

In [19]:
os.chdir(Defaults.STIM_DIR)

# other option is to hardcode name of csv
file_list = glob.glob('*')

# load in peele spreadsheet
df = pd.read_csv(file_list[0])

In [20]:
# add in new sentence columns

df['sentence'] = df['sentence'].str.extract(pat = "([A-Za-z ,']+)")
df['full_sentence'] = df['sentence'] + '' + df['target word']
                                          

In [21]:
# hardcode task parameters

num_sentences_per_block = 180
num_breaks_per_block = 2
trial_dur_ms = 10000
iti_dur = 500

# number of trials per block
trials_per_block = np.arange(start=1, stop=len(df) - 1, step=num_sentences_per_block)

In [22]:
# make gorilla spreadsheet

outname = Defaults.TARGET_DIR / f'all_blocks_{num_sentences_per_block}_trials.csv'

df_all = pd.DataFrame()
for i, block in enumerate(trials_per_block[:-1]):
    start_trial = trials_per_block[i]
    end_trial = trials_per_block[i+1]
    df_new = []
    df_new = df.loc[start_trial:end_trial]
    df_new['display'] = 'trial'
    df_new['iti_dur_ms'] = iti_dur
    df_new['trial_dur_ms'] = trial_dur_ms
    df_new['ShowProgressBar'] = 0
    
    # add instructions, breaks, and end display per block
    df_new = pd.concat([pd.DataFrame([{'display': 'instructions'}]), df_new], ignore_index=True, sort=False)
    df_new = df_new.append([{'display': 'end'}], ignore_index=True, sort=False)
    df_new.loc[breaks]['display'] = 'break'
    df_new.loc[breaks]['ShowProgressBar'] = 1
    
    # randomise blocks so that each participant gets a different
    # version of the experiment
    df_new['randomise_blocks'] = i+1
    
    # concat all blocks into one spreadsheet
    df_all = pd.concat([df_all, df_new], ignore_index=False, sort=False)
    
df_all.to_csv(outname, header=True, index=False)

NameError: name 'breaks' is not defined